In [1]:
import pandas as pd
from my_functions import *

In [2]:
matching = pd.read_csv('data/pt_data/Matching_File.txt',delimiter='\t')

In [3]:
matching.head(5)

,case_patid,case_pracid,case_gender,case_birthyear,case_index,match,control_patid,control_pracid,control_gender,control_birthyear,control_start,control_end
0,4923328,328,2,1916,01/07/2003,1,4355328,328,2,1916,01/01/2000,07/02/2009
1,1959216,216,1,1933,25/02/2011,1,6290216,216,1,1933,01/01/2000,01/06/2012
2,9271420,420,1,1917,20/06/2003,1,2770420,420,1,1917,02/01/2003,29/12/2009
3,548429,429,2,1925,11/05/2005,1,5473429,429,2,1925,01/01/2000,01/06/2012
4,14185081,81,2,1917,28/02/2005,1,5951081,81,2,1917,01/01/2000,27/01/2007


In [ ]:
feature_index=['patID','pracID','gender','birthyear','isCase','matchid','case_index','data_start','data_end']
features = pd.DataFrame(columns=feature_index)
features

,patID,pracID,gender,birthyear,isCase,matchid,case_index,data_start,data_end


In [ ]:
for index,row in matching.iterrows():
# first append the case
    index = len(features)
    features.loc[index,feature_index[0]]=row[0] #patID
    features.loc[index,feature_index[1]]=row[1] #pracID
    features.loc[index,feature_index[2]]=row[2] #gender
    features.loc[index,feature_index[3]]=row[3] #birthyear
    features.loc[index,feature_index[4]]=True #isCase
    features.loc[index,feature_index[5]]=row[6] #matchid
    features.loc[index,feature_index[6]]=row[4] #case_index
    features.loc[index,feature_index[7]]=row[10] #data_start
    features.loc[index,feature_index[8]]=row[11] #data_end
# now append the control
    index = len(features)
    features.loc[index,feature_index[0]]=row[6] #patID
    features.loc[index,feature_index[1]]=row[7] #pracID
    features.loc[index,feature_index[2]]=row[8] #gender
    features.loc[index,feature_index[3]]=row[9] #birthyear
    features.loc[index,feature_index[4]]=False #isCase
    features.loc[index,feature_index[5]]=row[0] #matchid
    features.loc[index,feature_index[6]]=row[4] #case_index
    features.loc[index,feature_index[7]]=row[10] #data_start
    features.loc[index,feature_index[8]]=row[11] #data_end

In [ ]:
features.tail(20)